In [9]:
import pandas as pd
import json
import getpass
import requests
import datetime
import os
import math


In [2]:
WB_TOKEN = os.getenv('WB_TOKEN')
headers = {'Authorization': WB_TOKEN}

In [6]:
cards_list_url = 'https://content-api.wildberries.ru/content/v2/get/cards/list'
body =             {
    "settings": {                      
        "cursor": {
            "limit": 100
        },
        "filter": {
            "withPhoto": 1,
            "objectIDs": [3979]
        }
    }
}
resp = requests.post(cards_list_url, headers=headers, json=body)


In [7]:
d = json.loads( resp.text)

In [8]:
d

{'cards': [{'nmID': 322201996,
   'imtID': 303629875,
   'nmUUID': '0194b654-afb6-7ff2-a107-871f0b168019',
   'subjectID': 3979,
   'subjectName': 'Коробки картонные',
   'vendorCode': 'box_222018_10',
   'brand': '',
   'title': 'Короб К-154, 180*180*80 мм 10 шт.',
   'needKiz': False,
   'photos': [{'big': 'https://basket-19.wbbasket.ru/vol3222/part322201/322201996/images/big/1.webp',
     'c246x328': 'https://basket-19.wbbasket.ru/vol3222/part322201/322201996/images/c246x328/1.webp',
     'c516x688': 'https://basket-19.wbbasket.ru/vol3222/part322201/322201996/images/c516x688/1.webp',
     'hq': 'https://basket-19.wbbasket.ru/vol3222/part322201/322201996/images/hq/1.webp',
     'square': 'https://basket-19.wbbasket.ru/vol3222/part322201/322201996/images/square/1.webp',
     'tm': 'https://basket-19.wbbasket.ru/vol3222/part322201/322201996/images/tm/1.webp'}],
   'dimensions': {'width': 72, 'height': 30, 'length': 52, 'isValid': True},
   'characteristics': [{'id': 90673, 'name': 'Шир

In [14]:
names = [n['name'] + "->" + str(n['required']) for n in d['data']]

In [23]:
def update_cards(api_key):
    """
    1. Получает список карточек (с фильтрами, если нужно).
    2. Изменяет характеристики и габариты для карточек, чей vendorCode начинается с 'box_'.
    3. Отправляет изменения методом /content/v2/cards/update.
    """
    
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    
    # ---------------------------
    # Шаг 1. Получаем список карточек
    # ---------------------------
    cards_list_url = "https://content-api.wildberries.ru/content/v2/get/cards/list"
    body = {
        "settings": {
            "cursor": {
                "limit": 100,  # Количество карточек на страницу
                'updatedAt': '2025-01-23T14:24:02.527842Z', 'nmID': 315307666,
            },
            "filter": {
                # Например, берем только те, у которых есть фото.
                # И предмет (subjectID) — "Коробки картонные" = 3979
                "withPhoto": 1,
                "objectIDs": [3979]
            }
        }
    }
    
    response = requests.post(cards_list_url, headers=headers, json=body)
    if response.status_code != 200:
        print(f"Ошибка при получении списка карточек: {response.status_code}")
        return
    
    data = response.json()
    if not data or "cards" not in data:
        print("Не удалось получить список карточек (нет ключа 'cards').")
        return
    
    print(data['cursor'])
    cards = data["cards"]
    
    # ---------------------------
    # Шаг 2. Подготавливаем карточки к обновлению
    # ---------------------------
    updated_cards = []
    
    for card in cards:
        vendor_code = card.get("vendorCode", "")
        if not vendor_code.startswith("box_"):
            # Пропускаем, если не начинается с 'box_'
            continue
        
        # Достаём характеристики и габариты
        characteristics = card.get("characteristics", [])
        dimensions = card.get("dimensions", {})
        
        # Ищем нужные значения:
        #  - Вес товара с упаковкой (г) (id=88952)
        #  - Количество предметов в упаковке (id=179792) (берём как int из первой строки массива)
        weight_with_pack_index = None
        quantity_in_pack_index = None
        
        for i, ch in enumerate(characteristics):
            if ch["id"] == 88952:
                weight_with_pack_index = i
            elif ch["id"] == 179792:
                quantity_in_pack_index = i
        
        # Меняем вес, если нашли обе характеристики
        if weight_with_pack_index is not None and quantity_in_pack_index is not None:
            old_weight = characteristics[weight_with_pack_index]["value"]  # 109 (int)
            quantity_list = characteristics[quantity_in_pack_index]["value"]  # ['10'] (list of strings)
            if quantity_list and len(quantity_list) > 0:
                try:
                    qty = int(quantity_list[0])  # '10' -> 10
                    # Умножаем старый вес на количество
                    new_weight = old_weight * qty
                    characteristics[weight_with_pack_index]["value"] = new_weight
                except ValueError:
                    pass  # если не получилось преобразовать в число, пропускаем
        
        # Меняем height в dimensions (уменьшаем в 10 раз)
        # heights обычно int, но на всякий случай приводим
        if "height" in dimensions:
            try:
                old_height = dimensions["height"]
                new_height = math.floor(old_height / 10)  # округлим вниз
                dimensions["height"] = new_height
            except (TypeError, ValueError):
                pass
        
        # Собираем обновлённый объект. Обратите внимание, что нужно
        # передавать все поля (title, brand, description, sizes, и т.д.)
        # которые уже есть в card, иначе данные сотрутся.
        
        # Если каких-то полей в card нет, (brand, description и т.д.),
        # можно передавать пустые строки, но обязательно указать dimensions,
        # characteristics, sizes (пусть даже пустой список).
        
        updated_card = {
            "nmID": card["nmID"],
            "vendorCode": card["vendorCode"],
            "brand": card.get("brand", ""),
            "title": card.get("title", ""),
            "description": card.get("description", ""),
            "dimensions": {
                "length": dimensions.get("length", 0),
                "width": dimensions.get("width", 0),
                "height": dimensions.get("height", 0)
            },
            "characteristics": characteristics,
            # sizes может отсутствовать в ответе. Если карточка без размеров —
            # передаем пустой массив. Если у вас есть данные по размерам, используйте их.
            "sizes": card.get("sizes", [])
        }
        
        updated_cards.append(updated_card)
    
    if not updated_cards:
        print("Нет карточек, требующих обновления.")
        return
    
    # ---------------------------
    # Шаг 3. Отправляем обновлённые карточки
    # ---------------------------
    update_url = "https://content-api.wildberries.ru/content/v2/cards/update"
    
    # Делаем запрос на обновление сразу всей порцией
    # (максимум 3000 карточек за раз)
    try:
        update_resp = requests.post(update_url, headers=headers, json=updated_cards)
        if update_resp.status_code == 200:
            update_data = update_resp.json()
            if not update_data.get("error", False):
                print("Карточки успешно обновлены.")
            else:
                print(f"Ошибка Wildberries: {update_data.get('errorText', 'Неизвестная ошибка')}")
        else:
            print(f"Ошибка ответа сервера при обновлении карточек: {update_resp.status_code}")
    except requests.RequestException as e:
        print(f"Ошибка при отправке запроса на обновление: {e}")


In [24]:
update_cards(WB_TOKEN)

{'nmID': 0, 'total': 0}
Нет карточек, требующих обновления.


In [12]:
data

NameError: name 'data' is not defined